<a href="https://colab.research.google.com/github/olonok69/projects_ai/blob/main/Cluster_embeddings_nomic_vs_mistral_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
! pip install -U bitsandbytes pysftp Faker -q

In [22]:
import json

In [114]:
cluster_numeclature_path = "/content/drive/MyDrive/data/CSM/cluster_numeculature.json"
session_nomic_embeddings_path = "/content/drive/MyDrive/data/CSM/embeddings_test_nomic.json"
session_mistral_embeddings_path = "/content/drive/MyDrive/data/CSM/embeddings_test_mistral.json"
old_session_mistral_embeddings_path = "/content/drive/MyDrive/data/CSM/session_embeddings_Mistral-7B-v0.1.json"
session_mistral_embeddings_path =  "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"


In [115]:
with open(cluster_numeclature_path, 'r') as f:
    numeculature_dict = json.load(f)

In [116]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
from tqdm import tqdm
import os
import gc

In [118]:
# del model
# del tokenizer
torch.cuda.empty_cache()
gc.collect()

0

# Mistral 7B

In [119]:


# model_id = "mistralai/Mixtral-8x7B-v0.1"
model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)



cluster_numeculature_embeddings = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor
    inputs = tokenizer(value, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to("cuda")

    # Forward pass, get all hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract hidden states of the second-to-last layer
    hidden_states = outputs.hidden_states
    second_to_last_layer = hidden_states[-2]

    cluster_numeculature_embeddings[key] = second_to_last_layer.mean(dim=1).cpu().numpy()

if os.path.exists('/content/drive/MyDrive/data/CSM/embeddings')!=True:
    os.makedirs('/content/drive/MyDrive/data/CSM/embeddings')

# Save embeddings to a file
with open(f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json', 'w', encoding='utf-8') as f:
    json.dump({k: v.tolist() for k, v in cluster_numeculature_embeddings.items()}, f, ensure_ascii=False, indent=4)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:01<00:00,  3.33it/s]


In [120]:
import numpy as np
import json

with open('/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_Mistral-7B-v0.1.json') as json_file:
    cluster_numeculature_dict_mistral = json.load(json_file)

with open("/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json") as json_file:
    session_embeddings_Mistral_7B_dict = json.load(json_file)

In [121]:
len(session_embeddings_Mistral_7B_dict.keys())

24837

In [123]:
len(session_embeddings_Mistral_7B_dict['BadgeId_22A5D99'][0])

4096

In [70]:
cluster_numeculature_dict_mistral.keys()

dict_keys(['Networking', 'Learning', 'Searching', 'Sourcing – Early', 'Sourcing – In Process', 'Sourcing – Deciding'])

In [124]:
len(cluster_numeculature_dict_mistral['Networking'][0])

4096

In [158]:
np.array(session_embeddings_Mistral_7B_dict['BadgeId_22A5D99']).shape

(1, 4096)

In [160]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(np.array(session_embeddings_Mistral_7B_dict['BadgeId_22A5D99']), np.array(cluster_numeculature_dict_mistral['Networking']))[0][0]

0.5523051873492829

In [162]:
def cosine_distance(A, B):
    dot_product = np.dot(A, B)
    magnitude_A = np.linalg.norm(A)
    magnitude_B = np.linalg.norm(B)
    return 1 - dot_product / (magnitude_A * magnitude_B)

def find_key_of_lowest_value(input_dict):
    # Initialize minimum value and corresponding key
    min_value = float('inf')
    min_key = None

    # Iterate through the dictionary
    for key, value in input_dict.items():
        # Update minimum value and key if a lower value is found
        if value < min_value:
            min_value = value
            min_key = key

    return min_key
def find_key_of_higher_value(input_dict):
    # Initialize minimum value and corresponding key
    min_value = float('-inf')
    min_key = None

    # Iterate through the dictionary
    for key, value in input_dict.items():
        # Update minimum value and key if a lower value is found
        if value > min_value:
            min_value = value
            min_key = key

    return min_key
def sort_dict_by_values(input_dict):
    # Sort the dictionary by its values in ascending order
    sorted_items = sorted(input_dict.items(), key=lambda x: x[1])
    return sorted_items

In [164]:
clusters={}
for badge_id, session_embeddings in session_embeddings_Mistral_7B_dict.items():
    cosine_distances = {}
    cosines_similarities = {}
    #print(badge_id)
    for cluster_group_name, cluster_numeculature in cluster_numeculature_dict_mistral.items():
        cosine_distances[cluster_group_name] = cosine_distance(np.squeeze(session_embeddings), np.squeeze(cluster_numeculature))
        cosines_similarities[cluster_group_name] = cosine_similarity(np.array(session_embeddings), np.array(cluster_numeculature))[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters.keys():
        clusters[badge_id] = closest_cluster_group_name_cos
    #print(f"Badge {badge_id}, closer distance {closest_cluster_group_name}, closer cosine {closest_cluster_group_name_cos}")
    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [165]:
len(clusters.keys())

24837

In [166]:
for key in list(clusters.keys())[0:5]:
  print(key, clusters[key])

BadgeId_22A5D99 Sourcing – In Process
BadgeId_22DIQKA Sourcing – In Process
BadgeId_22QZ49X Sourcing – In Process
BadgeId_237XLCW Sourcing – In Process
BadgeId_23BG2SF Sourcing – In Process


# LLama


In [167]:
del model

del tokenizer
torch.cuda.empty_cache()
gc.collect()

7573

In [168]:
torch.cuda.empty_cache()
gc.collect()

20

In [169]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [170]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_llama.json"

In [172]:
cluster_numeculature_embeddings_llama = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor
    inputs = tokenizer(value, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to("cuda")

    # Forward pass, get all hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Extract hidden states of the second-to-last layer
    hidden_states = outputs.hidden_states
    second_to_last_layer = hidden_states[-2]

    cluster_numeculature_embeddings_llama[key] = second_to_last_layer.mean(dim=1).cpu().numpy()

if os.path.exists('/content/drive/MyDrive/data/CSM/embeddings')!=True:
    os.makedirs('/content/drive/MyDrive/data/CSM/embeddings')

# Save embeddings to a file
with open(f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json', 'w', encoding='utf-8') as f:
    json.dump({k: v.tolist() for k, v in cluster_numeculature_embeddings_llama.items()}, f, ensure_ascii=False, indent=4)

100%|██████████| 6/6 [00:00<00:00,  7.52it/s]


In [173]:
f'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_{model_id.split("/")[1]}.json'

'/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_Llama-3.2-3B-Instruct.json'

In [174]:
with open('/content/drive/MyDrive/data/CSM/embeddings/cluster_numeculature_embeddings_Llama-3.2-3B-Instruct.json') as json_file:
    cluster_numeculature_dict_llama = json.load(json_file)

with open("/content/drive/MyDrive/data/CSM/badge/embeddings_badges_llama.json") as json_file:
    session_embeddings_LLama = json.load(json_file)

In [175]:
clusters3={}
for badge_id, session_embeddings in session_embeddings_LLama.items():
    cosine_distances = {}
    cosines_similarities = {}
    #print(badge_id)
    for cluster_group_name, cluster_numeculature in cluster_numeculature_dict_llama.items():
        #cosine_distances[cluster_group_name] = cosine_distance(np.squeeze(session_embeddings), np.squeeze(cluster_numeculature))
        cosines_similarities[cluster_group_name] = cosine_similarity(np.array(session_embeddings), np.array(cluster_numeculature))[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    #closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters3.keys():
        clusters3[badge_id] = closest_cluster_group_name_cos
    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [176]:
len(clusters3.keys())

24837

In [177]:
for key in list(clusters3.keys())[0:5]:
  print(key, clusters3[key])

BadgeId_22A5D99 Sourcing – In Process
BadgeId_22DIQKA Sourcing – In Process
BadgeId_22QZ49X Learning
BadgeId_237XLCW Sourcing – In Process
BadgeId_23BG2SF Sourcing – In Process


# Nomic

In [178]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/40b98394640e630d5276807046089b233113aa87/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

In [179]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_nomic.json"

In [180]:
with open(output_path) as json_file:
    session_embeddings_nomic_dict = json.load(json_file)

In [181]:
len(session_embeddings_nomic_dict['BadgeId_22A5D99'])


768

In [182]:
numeculature_dict


{'Networking': 'Networking: Visitors in this group are primarily focused on building professional relationships and expanding their business network. They attend events to meet industry peers, experts, and potential partners. Their interactions are more about establishing connections than immediate purchasing. They might be decision-makers or influencers in their organizations who value personal interactions and opportunities for collaboration.',
 'Learning': 'Learning: These attendees are motivated by educational opportunities. They seek to gain new insights, learn about industry trends, and enhance their knowledge and skills. They are likely to attend workshops, seminars, and keynote speeches. Their interest in products or services is secondary to their desire for professional development. They may represent a long-term opportunity as their increased knowledge could lead to future purchasing decisions.',
 'Searching': 'Searching (Gathering Information on Products or Services): Indivi

In [183]:
cluster_numeculature_embeddings_nomic = {}
for key, value in tqdm(numeculature_dict.items()):
    # Tokenize the input text and convert to tensor

    cluster_numeculature_embeddings_nomic[key] = model.encode(f'search_document: {value}')

100%|██████████| 6/6 [00:00<00:00, 45.18it/s]


In [184]:
cluster_numeculature_embeddings_nomic.keys()

dict_keys(['Networking', 'Learning', 'Searching', 'Sourcing – Early', 'Sourcing – In Process', 'Sourcing – Deciding'])

In [185]:
len(cluster_numeculature_embeddings_nomic['Networking'])

768

In [186]:
np.squeeze(cluster_numeculature_embeddings_nomic['Networking']).shape

(768,)

In [190]:
cosine_similarity(np.array(session_embeddings_nomic_dict['BadgeId_22A5D99']).reshape(1, -1), np.array(cluster_numeculature_embeddings_nomic['Networking']).reshape(1, -1))[0][0]

0.5118196638042893

In [192]:
clusters2={}
for badge_id, badge_embeddings in session_embeddings_nomic_dict.items():
    cosine_distances = {}
    cosines_similarities = {}
    #print(badge_id)
    for cluster_group_name, cluster_numeculature in cluster_numeculature_embeddings_nomic.items():
        #cosine_distances[cluster_group_name] = cosine_distance(np.squeeze(session_embeddings), np.squeeze(cluster_numeculature))
        cosines_similarities[cluster_group_name] = cosine_similarity(np.array(badge_embeddings).reshape(1, -1),
                                                                     np.array(cluster_numeculature).reshape(1, -1))[0][0]

    # print(json.dumps(cosine_distances, indent=2))
    #closest_cluster_group_name = find_key_of_lowest_value(cosine_distances)
    closest_cluster_group_name_cos = find_key_of_higher_value(cosines_similarities)
    if badge_id not in clusters2.keys():
        clusters2[badge_id] = closest_cluster_group_name_cos


    # Sort the dictionary and print the result
    # sorted_cosine_distances = sort_dict_by_values(cosine_distances)
    # print(">>>>>>")
    # for key, value in sorted_cosine_distances:
    #     print(f"{key}: {value}")
    # print(f"    closest cluster: {closest_cluster_group_name}")
    # print("<<<<<<")
    # input()

In [194]:
len(clusters2.values())

24837

In [193]:
for key in list(clusters2.keys())[0:5]:
  print(key, clusters2[key])

BadgeId_22A5D99 Networking
BadgeId_22DIQKA Networking
BadgeId_22QZ49X Networking
BadgeId_237XLCW Networking
BadgeId_23BG2SF Networking


In [195]:
labels = list(clusters2.values())

In [196]:
from collections import Counter
Counter(labels)

Counter({'Networking': 23309, 'Learning': 1527, 'Sourcing – In Process': 1})

In [197]:
labels_mistral = list(clusters.values())
Counter(labels_mistral)

Counter({'Sourcing – In Process': 24802, 'Sourcing – Deciding': 35})

In [198]:
labels_llama = list(clusters3.values())
Counter(labels_llama)

Counter({'Sourcing – In Process': 23315, 'Learning': 1522})

In [93]:
#clusters3